# **\[FRAUD\]** df02 accuracy 0.9707 f1은 망함

김보람  
2023-09-05

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import xgboost as xgb

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# embedding 
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder, AverageEmbedder, WeightedL1Embedder, WeightedL2Embedder

# gnn
import torch
import torch_geometric
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

`-` fraudTrain

In [6]:
fraudTrain = pd.read_csv("~/Desktop/fraudTrain.csv").iloc[:,1:]

In [7]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

`-` df02

In [12]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02 = df02.reset_index()

`-` df_toy

In [13]:
df_toy=df02[:5].copy()
df_toy.cc_num = pd.Series([1,1,1,2,2])
df_toy

`-` df_toy 에서 time_difference 구함

**고객1**

In [14]:
df_toy.iloc[0].trans_date_trans_time.value - df_toy.iloc[1].trans_date_trans_time.value

In [15]:
df_toy.iloc[0].trans_date_trans_time.value - df_toy.iloc[2].trans_date_trans_time.value

In [16]:
df_toy.iloc[1].trans_date_trans_time.value - df_toy.iloc[2].trans_date_trans_time.value

**고객2**

In [17]:
df_toy.iloc[3].trans_date_trans_time.value - df_toy.iloc[4].trans_date_trans_time.value

**고객1,2**

In [18]:
def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs(group.iloc[i].trans_date_trans_time.value - group.iloc[j].trans_date_trans_time.value)
            result.append([group.iloc[i].name, group.iloc[j].name, time_difference])
    return result

In [19]:
groups = df_toy.groupby('cc_num')
edge_index_list_plus = [compute_time_difference(group) for _, group in groups]
edge_index_list_plus_flat = [item for sublist in edge_index_list_plus for item in sublist]
edge_index_list_plus_nparr = np.array(edge_index_list_plus_flat)
edge_index_list_plus_nparr

`-` df02에서 time_difference 구함

In [20]:
# t1 = time.time()
# groups = df02.groupby('cc_num')
# edge_index_list_plus = [compute_time_difference(group) for _, group in groups]
# edge_index_list_plus_flat = [item for sublist in edge_index_list_plus for item in sublist]
# edge_index_list_plus_nparr = np.array(edge_index_list_plus_flat)
# np.save('edge_index_list_plus02.npy', edge_index_list_plus_nparr)
# t2 = time.time()
# t2-t1

In [162]:
groups = df02.groupby("cc_num")

In [11]:
edge_index_list_plus02[:,2] = (np.exp(-edge_index_list_plus02[:,2]/theta) != 1)*(np.exp(-edge_index_list_plus02[:,2]/theta))

In [12]:
edge_index_list_plus02

In [9]:
theta = edge_index_list_plus02[:,2].mean()

In [48]:
edge_index_list_plus02 = np.load('edge_index_list_plus02.npy').astype(np.float64)

In [49]:
edge_index_list_plus02[:,2] = (np.exp(-edge_index_list_plus02[:,2]/theta) != 1)*(np.exp(-edge_index_list_plus02[:,2]/theta))

In [50]:
edge_index_list_plus02

같은 cc_num별로.. 시간 차이를 계산했어.

In [176]:
weight = (np.exp(-edge_index_list_plus02[:,2]/theta) != 1)*(np.exp(-edge_index_list_plus02[:,2]/theta))

In [180]:
weight

자꾸 아래처럼 하면 3열의 값이 다 0나와가지고;; 이상하게 해봄

``` python
edge_index_list_plus02[:,2] = (np.exp(-edge_index_list_plus02[:,2]/theta) != 1)*(np.exp(-edge_index_list_plus02[:,2]/theta))
```

In [178]:
edge_index_list_plus02 = np.column_stack((edge_index_list_plus02, weight))

In [179]:
edge_index_list_plus02

In [181]:
edge_index_list_plus02 = np.delete(edge_index_list_plus02, 2, axis=1)

In [182]:
edge_index_list_plus02

In [183]:
edge_index_list_plus02.shape

In [184]:
edge_index_list_updated = edge_index_list_plus02.tolist()

In [186]:
np.array(edge_index_list_updated)[:,2].mean()

In [187]:
mm = np.array(edge_index_list_updated)[:,2].mean()

In [188]:
selected_edges = [(int(row[0]), int(row[1])) for row in edge_index_list_updated if row[2] > mm]
edge_index_selected = torch.tensor(selected_edges, dtype=torch.long).t()
edge_index_selected.shape

## tr/test

In [26]:
df02_tr,df02_test = sklearn.model_selection.train_test_split(df02, random_state=42)

In [27]:
df02_tr.shape, df02_test.shape

In [28]:
N = len(df02)
train_mask = [i in df02_tr.index for i in range(N)]
test_mask = [i in df02_test.index for i in range(N)]

In [29]:
train_mask = np.array(train_mask)
test_mask = np.array(test_mask)

In [31]:
train_mask.shape, test_mask.shape

## data

In [192]:
x = torch.tensor(df02['amt'], dtype=torch.float).reshape(-1,1)
y = torch.tensor(df02['is_fraud'],dtype=torch.int64)
data = torch_geometric.data.Data(x=x, edge_index = edge_index_selected, y=y, train_mask = train_mask, test_mask = test_mask)

In [193]:
data

In [194]:

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [195]:
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(400):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [220]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(np.array(data.test_mask).sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.9707

In [209]:
predicted_labels = pred[data.test_mask]
true_labels = data.y[data.test_mask]

In [210]:
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Precision: 0.4854
Recall: 0.5000
F1 Score: 0.4926

/home/coco/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))